In [1]:
from tensorflow import keras
import numpy as np

2022-11-02 18:45:53.838458: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

In [3]:
text = open(path).read().lower()
print('Corpus length:', len(text))

Corpus length: 600893


In [4]:
maxlen = 60
step = 3

sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 57
Vectorization...


In [5]:
from tensorflow.keras import layers
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

2022-11-02 18:46:03.080231: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-02 18:46:03.080688: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-11-02 18:46:04.053844: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-02 18:46:04.054112: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemoryBandwidth: 607.97GiB/s
2022-11-02 18:46:04.054124: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-11-02 18:46:04.054879: I tensorflow/stream_executor/platform/default/dso_loade

In [6]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [7]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [8]:
import random
import sys

for epoch in range(1, 60):
    
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)
        
        for i in range(400):
            
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            generated_text += next_char
            generated_text = generated_text[1:]
            sys.stdout.write(next_char)

epoch 1


2022-11-02 18:46:17.991172: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-11-02 18:46:18.008948: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3600000000 Hz
2022-11-02 18:46:18.687734: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-11-02 18:46:18.838117: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


1565/1565 [==============================] - 24s 14ms/step - loss: 2.2510
--- Generating with seed: ": whereas here, no less than in the hippodrome, it
is necess"
------ temperature: 0.2
: whereas here, no less than in the hippodrome, it
is necessity of the prefiner and still and the still the still the sense of the still and all the sense of the himself, and the thinker the still the still the sense of the still of the prefiners of the still and the sense of the still and the presention the the many and and the expecience of the the religious the still of the still and the presention and conception the presention the still and the sense o------ temperature: 0.5
tion and conception the presention the still and the sense of the still the prefiners and regards and his the sense of the still of the the thinker the intellect the emecanion the may rea, and the will this here a greeked and perpossible be the presentificition and comes free from this in the still more it litt and the still an

other distief ctance are, bet i neweven: as any fell in
hithelsued--love himself whow so8 his
ascertains of be. actualial pepsingly of his verded toned
and the other
ofusts
prelocited;ed must impoedled to pined drity. here odeved countly
a pepoch 5
1565/1565 [==============================] - 22s 14ms/step - loss: 1.4513
--- Generating with seed: "rd to him, as
free from responsibility as if plants and ston"
------ temperature: 0.2
rd to him, as
free from responsibility as if plants and stone the spirit and more and soul a man are with the specialize to and in the sense of the sense of the most sacrifice and seek of the standard of the sense of the sense of the most and more the sense of the sense of the sense of the more spirit and soul as a more for the more includification of the sense of the sense of the inclusion of the spirit of the sense of the same the spirit of the sense of ------ temperature: 0.5
 spirit of the sense of the same the spirit of the sense of all the fast of the 

in all howed ihoe and hear our 
ard, us as sleastsmelys" sympathy, thoushe, surrowed aed
chresing thro, that he
cormines,
thotesess
to good as theing by aniuph -inapaces, schetical
atprys
a condust for
emernartice and knowsermely betawa's out
ourrentible to-getsect. than pivint--nachver his
schoul, the sease peruinath,
the cyr "quebtred tew no givet--    what it is compri-ait andrementchares", on three as resmest may, tepoch 9
1565/1565 [==============================] - 22s 14ms/step - loss: 1.3866
--- Generating with seed: "ed enormous proportions--to such enormous proportions, indee"
------ temperature: 0.2
ed enormous proportions--to such enormous proportions, indeed the strength and substity of the strength of the spirit of the spirit to the spirit the strength and strength and conscience and so is a most spirit of the strength of the spirit in the spirit to the strength of the spirit of the spirit to the spirit to the strength of the spirit of the strength and the fact the streng

...

                                                                                                                                                                                                                          the presence of the same the same conscience of the most action, when the most into the conscience of the character, which is also all of individual. whene has to be all every reason the predicted of the most thing as else
places and possible------ temperature: 1.0
 the predicted of the most thing as else
places and possible
of lis honest oblir as what has not be mainly of intellect of himself. it is only to be diong and wealinessm of the
contain fool, as if over hand, with "has i race and for of him is inclusion, at dions, as
regarded, "galust on the
music, as now thewing curre, artwise and, i find--finally amplin for this ideas would be renalus more,
not inno good like
it.

12. as last which he is really who are no------ temperature: 1.2
nno good like
it.

12. as last w

to place the greatness of man, the conception of "greatness, the world and sublight to the consequently and concealed that the contention that the waring and the same the subtle truth and moral and suffering the spirit to precisely the same to the strength and propertion that the spirit to the same the problemment and properte which is the concerning the same the same the same the spirit and the same the same the desires that the same the spirit that t------ temperature: 0.5
he same the same the desires that the same the spirit that the spection that is any case of the same truth to be attain to the indigners of its process and flow, and should be not aid that they consequence and the most end allow the fact that the last who
has a woman a sure to the faith to precisely because that it is because that he caste
of nothing, which is art and good and sure as the problemh and special power which is looks us to the fact that the be------ temperature: 1.0
 and special power which is looks us

short, darifourns; fyarnes hithertoral disclugier than to seclet communism, for above a dispepoch 51
1565/1565 [==============================] - 22s 14ms/step - loss: 1.2547
--- Generating with seed: "rian, and besides, poet, and collector, and traveler, and
ri"
------ temperature: 0.2
rian, and besides, poet, and collector, and traveler, and
rightly and conscience of the same time in the moral and the belief in the same time the strength and superstition of the same tendency of the world and the strength and the same truthful and superstitious superstity and man and superior of the fact the same supersofor of the same truthful and conscience of the same tendency of the same time in the same time as the strength and superior
and supersti------ temperature: 0.5
e in the same time as the strength and superior
and superstitious father--as a delight of subtle the subject the same instinct of the same a man has a god as a god and the world of the most presence of mankind with a person the 

time, and that
one araia frackley. the delys, who
takes fhow. ske, art sought as much loso the as eortc enjoogenta
abserved at forweaps, ma additionally intelre all the
ssame new, as lrews, unexternal ants! onts a harmen
to
theory threxacctous something divine rare olwament endury, who gaver loke
very fre itdemplace,
seech forth. it were into epoch 55
1565/1565 [==============================] - 22s 14ms/step - loss: 1.2495
--- Generating with seed: ", the entire movement of the world
stopped short, and an all"
------ temperature: 0.2
, the entire movement of the world
stopped short, and an alled the most desire the spirit of the conscious existence of the spirit of the superior the spirit the spirit in the spirit of the highest desires to the fact that it is not an antithesis of the spirit in the faithing of the spirit in the sense of the spirit," and it is not the same successful to the best obey and more belief of the conscious and superior and superfuce of the history of the most s

to moral-exist, into another, to certain externaluties, though the demawirs for their middle in philosopher in veluvise, even the child comprehensibent, like literary dow------ temperature: 1.2
 veluvise, even the child comprehensibent, like literary down
classes as first has tets gumbeance the grees--and
different refugudy, one
"under they live incarn"--" here of domarus (and another-glower) ourdual gressing with  or drefits, whhe is in
i
tyetiful bad so peculiaw should rahed"
apparending even in the perioding. the verity in spiedle influele a tendination, or quiceb thorifici": all beside to be the syntheme of his tandow his german comporturay tepoch 59
1565/1565 [==============================] - 22s 14ms/step - loss: 1.2442
--- Generating with seed: "lity can first be laid only when a stronger individuality or"
------ temperature: 0.2
lity can first be laid only when a stronger individuality or soul as a such a delicate of the subliment, the present soul the present the sublighter a